In [1]:
import warnings
from typing import List

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from autumn.core.project import get_project
from autumn.settings import Region, Models
from autumn.core import inputs
from autumn.core.utils.display import pretty_print
from autumn.core.inputs.tb_camau import queries
from autumn.core.inputs import get_death_rates_by_agegroup
import pathlib

In [2]:
warnings.filterwarnings("ignore")
pd.options.plotting.backend = "plotly"

In [3]:
region = Region.CAMAU
model = Models.TBD2
p = get_project(model, region, reload=True)
baseline_params = p.param_set.baseline
plots = p.plots
iso3 = baseline_params['country']['iso3']
source_iso3 = baseline_params['age_mixing']['source_iso3']
age_adjust = baseline_params['age_mixing']['age_adjust']
age_string_map = {
    0: "0-4",
    5: "5-14",
    15: "15-34",
    35: "35-49",
    50: "50+",
}

## Population

In [4]:
modelled_age_groups = baseline_params["age_breakpoints"]
print(f"Modelled age groups are {modelled_age_groups}")

Modelled age groups are [0, 5, 15, 35, 50]


In [5]:
csv_path = pathlib.Path('camau.csv')

In [6]:
pop_df = pd.read_csv(csv_path)
pop_df = pop_df.set_index(['year'])
pop_df['population'].plot()

### Birth rate

In [7]:
birth_rates, years = inputs.get_crude_birth_rate("VNM")
birth_rates = [b / 1000.0 for b in birth_rates]  # Birth rates are provided / 1000 population
birth_rates_series = pd.Series(birth_rates, index=years)
br_fig= px.line(birth_rates_series)
br_fig.update_traces(mode='markers+lines')
br_fig.update_layout(
    title="Crude birth rate of Ca Mau",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Crude birth rate",
    showlegend=False,
)

### Death rate

In [8]:
# Currently used in model:
death_rates_from_db = get_death_rates_by_agegroup(modelled_age_groups, "VNM")
death_rates_df = pd.DataFrame(death_rates_from_db[0], index=death_rates_from_db[1])
life_exp_at_age_df = 1.0 / death_rates_df
death_rates_df.plot()
life_exp_at_age_df.plot(log_y=True)

In [9]:
# Query and visualize the crude birth rate from database
years, death_rates = queries.get_camau_death_rate()
death_rates = [b / 1000.0 for b in death_rates]  # Birth rates are provided / 1000 population
death_rates_series = pd.Series(death_rates, index=years)
dr_fig= px.line(death_rates_series)
dr_fig.update_traces(mode='markers+lines')
dr_fig.update_layout(
    title="Crude death rate of Ca Mau",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Crude death rate",
    showlegend=False,
)

### Mixing matrix

In [10]:
print(f"Target region: {region.upper()} ({iso3})")
print(f"Proxy country: {source_iso3}")
print("Always age-adjusted to target population")

Target region: CAMAU (VNM)
Proxy country: VNM
Always age-adjusted to target population


In [11]:
update_params = {
    'cdr_adjustment': 0.781297481378072,
    'contact_rate': 0.009970201117109802,
    'infect_death_rate_dict.smear_negative': 0.024995732286672893,
    'infect_death_rate_dict.smear_positive': 0.4374233558323563,
    'progression_multiplier': 0.12206487367166488,
    'rr_infection_latent': 0.48193872958821177,
    'rr_infection_recovered': 0.23345947728785726,
    'self_recovery_rate_dict.smear_positive': 0.18727973587244693,
    'self_recovery_rate_dict.smear_negative': 0.0882238241945665,
    'start_population_size': 330000
}
params = baseline_params.update(update_params,calibration_format=True)
model_0  =  p.build_model(params.to_dict())
model_0.run(params.to_dict())
derived_df_0 = model_0.get_derived_outputs_df()

In [12]:
model_0._stratifications

[Stratification: age, Stratification: gender, Stratification: organ]

In [13]:
model_0._mixing_matrices

[array([[ 398.43289672,  261.82020387,  643.68286218,  401.62199159,
          356.13449939],
        [ 165.78966683,  881.63067677,  532.84120554,  550.75979227,
          285.62836724],
        [ 231.75164317,  311.38983781,  915.52884268,  673.30894113,
          664.14577066],
        [ 141.94492435,  310.88835505,  786.13676958, 1134.31076003,
          938.03403291],
        [  67.30073632,  170.46333134,  647.30153978, 1018.81243422,
         1763.57657715]])]

In [14]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [15]:
latent_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="percentage_latent",
)
latent_2 = px.scatter(x= plots['percentage_latent']['times'], y = plots['percentage_latent']['values'])
latent_2.update_traces(marker=dict(color="red"))
latent_plot = go.Figure(
    data=latent_1.data + latent_2.data,
)
latent_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
latent_plot.show()

In [16]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="notifications",
)
notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
notif_2.update_traces(marker=dict(color="red"))
notif_plot = go.Figure(
    data=notif_1.data + notif_2.data,
)
notif_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
)
notif_plot.show()

In [17]:
inci_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="incidence",
)
inci_plot.update_layout(
    title="", title_x=0.5, xaxis_title="Year", yaxis_title="Incidence per 100,000 population"
)
inci_plot.show()

In [18]:
prev_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="prevalence_infectious",
)
prev_plot.update_layout(
    title="", title_x=0.5, xaxis_title="Year", yaxis_title="Prevalance per 100,000 population"
)
prev_plot.show()

In [19]:
derived_df_0

,cumulative_deaths,cumulative_diseased,incidence,incidence_early,incidence_late,notifications,percentage_latent,prevalence_infectious,total_population
1900.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.015687,0.000000,0.303030,3.300000e+05
1900.1,0.000000,0.000000,0.547569,1.806555,0.000088,1.004449,0.000323,0.298977,3.299391e+05
1900.2,0.000000,0.000000,1.388268,4.579299,0.000405,0.996582,0.000623,0.302896,3.298861e+05
1900.3,0.000000,0.000000,1.851945,6.107514,0.000961,1.010634,0.000911,0.310701,3.298411e+05
1900.4,0.000000,0.000000,2.132679,7.031971,0.001690,1.035891,0.001194,0.320513,3.298039e+05
...,...,...,...,...,...,...,...,...,...
2019.6,116060.715386,322962.949230,174.596035,1468.373107,652.298670,1422.478580,35.854482,21.911671,1.214616e+06
2019.7,116109.991520,323175.055421,174.597477,1470.400077,650.661836,1420.769521,35.764242,21.880075,1.214830e+06
2019.8,116159.220825,323387.191709,174.591817,1472.334248,649.028629,1419.273982,35.674202,21.851689,1.215041e+06
2019.9,116208.410064,323599.280567,174.522603,1473.489532,647.399052,1417.948833,35.584334,21.825153,1.215252e+06
